In [1]:
sealed interface Entry {
    enum class BinOp: Entry { ADD, MULT }
    data class Value(val number: Int): Entry
}

interface ICalculator {
    fun appendEntry(e: Entry): Boolean // input either BinOp or Vaule
        // appends legal entry input and return true (returns false for illegal input)
        // updates last calculated value if possible
    fun removeEntry(): Boolean // push the x button
        // removes the last input entry and return true (returns false if it is not possible)
        // updates last calculated value if necessary
    fun calculate(): Entry.Value?    // push the = button
        // calculate current entry and returns the result value (return null when it fails) 
        // add to history
        // clear current entries and nullify last calculated value
    fun lastValue(): Entry.Value? // last calculated value
    fun entries(): List<Entry> // current entries
    fun history(): List<Pair<Entry, Entry.Value>> // history of calculations
    fun clearEntries() // clear the current entries
    fun clearHistory() // clear history
}

In [2]:
// ... class Calculator: ICalculator ...